In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
tqdm.pandas()

In [139]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [140]:
print("Shape of Train: ", train.shape)
train.head()

Shape of Train:  (450000, 20)


,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [141]:
train['cancelled'].value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

Feature Ideas:
- Time to Allot Order to Rider
- Time taken by rider to accept order
- Time for Food Preparation
- Time taken for delivery from time of order
- Check if there is an overlap between rider ID in train and test, if there is, add a column for rider cancellation rate
- If clustering happens for time of day of cancellation, add as a feature
- Ratio of first mile to last mile distance

Cleaning Ideas:
- lifetime_order_count change as per date
- interpolate based on rider_id for session time

Future Ideas:
- Subsample first and then smote in order to prevent smote from messing up

In [142]:
train[train['accept_time'].isna()].head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
1700,2021-01-26 06:35:16,558453,2021-01-26 00:00:00,2021-01-26 06:35:18,NaN,NaN,NaN,15903,1.8026,8.55,50.0,50.0,1,0.0,1216.0,NaN,NaN,NaN,51.300000,2021-01-26 06:40:17
4066,2021-01-26 07:20:44,560819,2021-01-26 00:00:00,2021-01-26 07:20:45,NaN,NaN,NaN,3430,1.6112,4.29,29.0,29.0,1,0.0,190.0,NaN,NaN,NaN,52.700000,2021-01-26 07:21:20
9296,2021-01-26 08:58:12,566050,2021-01-26 00:00:00,2021-01-26 08:58:41,NaN,NaN,NaN,20621,2.8689,4.07,NaN,NaN,1,NaN,917.0,NaN,NaN,NaN,120.733333,2021-01-26 09:00:35
13365,2021-01-26 12:42:36,570118,2021-01-26 00:00:00,2021-01-26 12:42:37,NaN,NaN,NaN,10605,0.7966,3.14,11.0,11.0,1,0.0,1546.0,NaN,NaN,NaN,478.333333,2021-01-26 12:47:37
13429,2021-01-26 12:45:42,570182,2021-01-26 00:00:00,2021-01-26 12:45:43,NaN,NaN,NaN,3585,0.9797,7.79,67.0,65.0,1,2.0,144.0,NaN,NaN,NaN,325.666667,2021-01-26 12:47:38


In [143]:
print("Shape of Test: ", test.shape)
test.head()

Shape of Test:  (144844, 16)


,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333


In [144]:
len(set(train['rider_id']).intersection(set(test['rider_id'])))

11652

In [145]:
len(test['rider_id'].unique())

13682

In [146]:
train[train['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,2021-02-03 06:55:20,2021-02-03 07:12:18,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,NaN
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,2021-02-03 14:06:26,2021-02-03 14:46:21,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,NaN
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,2021-02-03 14:15:04,2021-02-03 14:33:03,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,NaN
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,2021-02-03 16:31:13,2021-02-03 16:40:45,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,NaN
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,2021-02-03 16:49:22,2021-02-03 16:58:00,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,NaN
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,2021-02-03 17:28:24,2021-02-03 17:35:16,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,NaN
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,2021-02-03 17:41:59,2021-02-03 18:04:50,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,NaN
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,2021-02-04 14:29:13,2021-02-04 14:41:52,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,NaN
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,2021-02-04 15:15:29,2021-02-04 15:23:42,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,NaN
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,2021-02-04 15:55:55,2021-02-04 16:04:07,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,NaN


In [147]:
test[test['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
5451,2021-02-06 13:42:02,135683,2021-02-06 00:00:00,2021-02-06 13:42:48,2021-02-06 13:44:05,1,0.669800,4.93,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,600.333333
17324,2021-02-06 16:02:40,147554,2021-02-06 00:00:00,2021-02-06 16:02:59,2021-02-06 16:03:42,1,1.324600,0.54,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,740.966667
18092,2021-02-06 16:11:36,148323,2021-02-06 00:00:00,2021-02-06 16:11:52,2021-02-06 16:13:21,1,0.029638,0.23,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,749.900000
23756,2021-02-06 17:19:38,153986,2021-02-06 00:00:00,2021-02-06 17:22:20,2021-02-06 17:22:24,1,1.311700,4.69,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,817.933333
32469,2021-02-07 08:03:10,86420,2021-02-07 00:00:00,2021-02-07 08:03:10,2021-02-07 08:04:41,1,0.965400,0.19,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,262.700000
41660,2021-02-07 13:33:05,95611,2021-02-07 00:00:00,2021-02-07 13:33:06,2021-02-07 13:33:15,1,2.511500,0.11,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,574.283333
44538,2021-02-07 13:58:36,98489,2021-02-07 00:00:00,2021-02-07 13:58:38,2021-02-07 13:58:50,1,1.551600,3.87,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,599.800000
46696,2021-02-07 14:19:07,100647,2021-02-07 00:00:00,2021-02-07 14:19:08,2021-02-07 14:19:18,1,0.929400,5.24,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,620.316667
50516,2021-02-07 15:21:35,104468,2021-02-07 00:00:00,2021-02-07 15:22:01,2021-02-07 15:23:02,1,0.322100,3.08,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,682.783333
58446,2021-02-07 16:47:26,112397,2021-02-07 00:00:00,2021-02-07 16:47:27,2021-02-07 16:47:44,1,1.536600,1.15,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,768.633333


In [148]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

In [149]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144844 entries, 0 to 144843
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            144844 non-null  object 
 1   order_id              144844 non-null  int64  
 2   order_date            144844 non-null  object 
 3   allot_time            144844 non-null  object 
 4   accept_time           144776 non-null  object 
 5   rider_id              144844 non-null  int64  
 6   first_mile_distance   144844 non-null  float64
 7   last_mile_distance    144844 non-null  float64
 8   alloted_orders        140071 non-null  float64
 9   delivered_orders      139960 non-null  float64
 10  undelivered_orders    139960 non-null  float64
 11  lifetime_order_count  144066 non-null  float64
 12  reassignment_method   4632 non-null    object 
 13  reassignment_reason   4635 non-null    object 
 14  reassigned_order      4635 non-null    float64
 15  

In [150]:
drop_cols = ['pickup_time', 'delivered_time', 'cancelled_time']
train.drop(drop_cols, axis = 1, inplace = True)

In [151]:
def time_diff(time_list_1, time_list_2):
    hour_diff = (int(time_list_1[0]) - int(time_list_2[0]))*3600
    min_diff = (int(time_list_1[1]) - int(time_list_2[1]))*60
    sec_diff = (int(time_list_1[2]) - int(time_list_2[2]))
    return hour_diff + min_diff + sec_diff
    
def create_time_features(row):
    
    order_times = row['order_time'].split(' ')[1].split(':')
    allot_times = row['allot_time'].split(' ')[1].split(':')
    if row['accept_time'] == row['accept_time']:
        accept_times = row['accept_time'].split(' ')[1].split(':')
    
    row['rider_allot_time'] = time_diff(allot_times, order_times)
    try:
        row['rider_accept_time'] = time_diff(accept_times, allot_times)
    except:
        row['rider_accept_time'] = -1
        
    return row

In [152]:
def apply_func(df):
    df = df.progress_apply(create_time_features, axis = 1)
    return df

def parallelize_dataframe(df, func, n_cores = 10):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [153]:
train = parallelize_dataframe(train, apply_func)
test = parallelize_dataframe(test, apply_func)

100%|████████████████████████████████████████████████████████████| 14485/14485 [00:30<00:00, 479.07it/s]


In [154]:
more_drop = ['order_time', 'order_date', 'allot_time',
             'accept_time', 'rider_id']

train.drop(more_drop, axis = 1, inplace = True)
test.drop(more_drop, axis = 1, inplace = True)

In [155]:
from sklearn.preprocessing import LabelEncoder
text_fill = ['reassignment_method', 'reassignment_reason']
num_fill = ['reassigned_order', 'session_time', 'alloted_orders',
       'delivered_orders', 'undelivered_orders', 'lifetime_order_count',]

for col in text_fill:
    le = LabelEncoder()
    train[col] = train[col].fillna('Missing')
    train[col] = le.fit_transform(train[col])
    test[col] = test[col].fillna('Missing')
    test[col] = le.transform(test[col])

for col in num_fill:
    train[col] = train[col].fillna(-1)
    test[col] = test[col].fillna(-1)

In [156]:
train['reassigned_order'].value_counts()

-1.0    436247
 1.0     13753
Name: reassigned_order, dtype: int64

In [118]:
X_data = train.drop(['cancelled', 'order_id'], axis = 1)
y_data = train['cancelled']

kfold = StratifiedKFold(random_state = 10, n_splits = 10, shuffle = True)
splits = kfold.split(X_data, y_data)

sm = SMOTE(random_state = 10, k_neighbors = 7, n_jobs = -1)

roc_scores = []
Models = []

for Train, Test in splits:
    X_Train, X_Val, Y_Train, Y_Val = X_data.iloc[Train], X_data.iloc[Test], y_data.iloc[Train], y_data.iloc[Test]
    xgb = XGBClassifier(n_estimators = 8000, max_depth = 8, n_jobs = -1,
                        learning_rate = 0.05, reg_lambda = 0.1, random_state = 10)
    X_Train, Y_Train = sm.fit_resample(X_Train, Y_Train)
    xgb.fit(X_Train, Y_Train, early_stopping_rounds = 200,
            eval_set = [(X_Train, Y_Train), (X_Val, Y_Val)],
            eval_metric = 'logloss', verbose = True)
    pred = xgb.predict(X_Val)
    
    roc_scores.append(roc_auc_score(pred, Y_Val))
    Models.append(xgb)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-logloss:0.66429	validation_1-logloss:0.66462
[1]	validation_0-logloss:0.63813	validation_1-logloss:0.63839
[2]	validation_0-logloss:0.61421	validation_1-logloss:0.61405
[3]	validation_0-logloss:0.59234	validation_1-logloss:0.59165
[4]	validation_0-logloss:0.57244	validation_1-logloss:0.57095
[5]	validation_0-logloss:0.55410	validation_1-logloss:0.55164
[6]	validation_0-logloss:0.53727	validation_1-logloss:0.53372
[7]	validation_0-logloss:0.52145	validation_1-logloss:0.51680
[8]	validation_0-logloss:0.50699	validation_1-logloss:0.50106
[9]	validation_0-logloss:0.49355	validation_1-logloss:0.48651
[10]	validation_0-logloss:0.48100	validation_1-logloss:0.47266
[11]	validation_0-logloss:0.46939	validation_1-logloss:0.45971
[12]	validation_0-logloss:0.45871	validation_1-logloss:0.44767
[13]	validation_0-logloss:0.44862	validation_1-logloss:0.43631
[14]	validation_0-logloss:0.43927	validation_1-logloss:0.42568
[15]	validation_0-logloss:0.43065	validation_1-logloss:0.41565
[1

[130]	validation_0-logloss:0.25362	validation_1-logloss:0.22378
[131]	validation_0-logloss:0.25333	validation_1-logloss:0.22361
[132]	validation_0-logloss:0.25311	validation_1-logloss:0.22347
[133]	validation_0-logloss:0.25275	validation_1-logloss:0.22310
[134]	validation_0-logloss:0.25223	validation_1-logloss:0.22276
[135]	validation_0-logloss:0.25191	validation_1-logloss:0.22247
[136]	validation_0-logloss:0.25160	validation_1-logloss:0.22231
[137]	validation_0-logloss:0.25092	validation_1-logloss:0.22195
[138]	validation_0-logloss:0.25066	validation_1-logloss:0.22173
[139]	validation_0-logloss:0.25051	validation_1-logloss:0.22162
[140]	validation_0-logloss:0.25004	validation_1-logloss:0.22131
[141]	validation_0-logloss:0.24926	validation_1-logloss:0.22092
[142]	validation_0-logloss:0.24902	validation_1-logloss:0.22077
[143]	validation_0-logloss:0.24873	validation_1-logloss:0.22046
[144]	validation_0-logloss:0.24863	validation_1-logloss:0.22033
[145]	validation_0-logloss:0.24844	valid

[259]	validation_0-logloss:0.21856	validation_1-logloss:0.19803
[260]	validation_0-logloss:0.21847	validation_1-logloss:0.19796
[261]	validation_0-logloss:0.21834	validation_1-logloss:0.19792
[262]	validation_0-logloss:0.21803	validation_1-logloss:0.19776
[263]	validation_0-logloss:0.21782	validation_1-logloss:0.19763
[264]	validation_0-logloss:0.21751	validation_1-logloss:0.19739
[265]	validation_0-logloss:0.21711	validation_1-logloss:0.19707
[266]	validation_0-logloss:0.21683	validation_1-logloss:0.19689
[267]	validation_0-logloss:0.21673	validation_1-logloss:0.19678
[268]	validation_0-logloss:0.21666	validation_1-logloss:0.19672
[269]	validation_0-logloss:0.21658	validation_1-logloss:0.19668
[270]	validation_0-logloss:0.21649	validation_1-logloss:0.19662
[271]	validation_0-logloss:0.21642	validation_1-logloss:0.19655
[272]	validation_0-logloss:0.21634	validation_1-logloss:0.19649
[273]	validation_0-logloss:0.21608	validation_1-logloss:0.19634
[274]	validation_0-logloss:0.21575	valid

[388]	validation_0-logloss:0.18307	validation_1-logloss:0.17205
[389]	validation_0-logloss:0.18247	validation_1-logloss:0.17162
[390]	validation_0-logloss:0.18206	validation_1-logloss:0.17128
[391]	validation_0-logloss:0.18140	validation_1-logloss:0.17085
[392]	validation_0-logloss:0.18109	validation_1-logloss:0.17058
[393]	validation_0-logloss:0.18062	validation_1-logloss:0.17027
[394]	validation_0-logloss:0.18032	validation_1-logloss:0.17011
[395]	validation_0-logloss:0.17997	validation_1-logloss:0.16994
[396]	validation_0-logloss:0.17979	validation_1-logloss:0.16983
[397]	validation_0-logloss:0.17943	validation_1-logloss:0.16952
[398]	validation_0-logloss:0.17897	validation_1-logloss:0.16920
[399]	validation_0-logloss:0.17878	validation_1-logloss:0.16906
[400]	validation_0-logloss:0.17859	validation_1-logloss:0.16895
[401]	validation_0-logloss:0.17849	validation_1-logloss:0.16889
[402]	validation_0-logloss:0.17845	validation_1-logloss:0.16884
[403]	validation_0-logloss:0.17817	valid

...

[3826]	validation_0-logloss:0.01615	validation_1-logloss:0.06312
[3827]	validation_0-logloss:0.01614	validation_1-logloss:0.06311
[3828]	validation_0-logloss:0.01613	validation_1-logloss:0.06311
[3829]	validation_0-logloss:0.01613	validation_1-logloss:0.06312
[3830]	validation_0-logloss:0.01613	validation_1-logloss:0.06312
[3831]	validation_0-logloss:0.01611	validation_1-logloss:0.06312
[3832]	validation_0-logloss:0.01610	validation_1-logloss:0.06312
[3833]	validation_0-logloss:0.01610	validation_1-logloss:0.06312
[3834]	validation_0-logloss:0.01610	validation_1-logloss:0.06311
[3835]	validation_0-logloss:0.01609	validation_1-logloss:0.06311
[3836]	validation_0-logloss:0.01609	validation_1-logloss:0.06311
[3837]	validation_0-logloss:0.01608	validation_1-logloss:0.06310
[3838]	validation_0-logloss:0.01607	validation_1-logloss:0.06310
[3839]	validation_0-logloss:0.01606	validation_1-logloss:0.06310
[3840]	validation_0-logloss:0.01606	validation_1-logloss:0.06310
[3841]	validation_0-loglo

[3953]	validation_0-logloss:0.01542	validation_1-logloss:0.06307
[3954]	validation_0-logloss:0.01541	validation_1-logloss:0.06307
[3955]	validation_0-logloss:0.01540	validation_1-logloss:0.06306
[3956]	validation_0-logloss:0.01540	validation_1-logloss:0.06306
[3957]	validation_0-logloss:0.01540	validation_1-logloss:0.06306
[3958]	validation_0-logloss:0.01540	validation_1-logloss:0.06306
[3959]	validation_0-logloss:0.01539	validation_1-logloss:0.06306
[3960]	validation_0-logloss:0.01538	validation_1-logloss:0.06306
[3961]	validation_0-logloss:0.01538	validation_1-logloss:0.06306
[3962]	validation_0-logloss:0.01537	validation_1-logloss:0.06306
[3963]	validation_0-logloss:0.01537	validation_1-logloss:0.06307
[3964]	validation_0-logloss:0.01536	validation_1-logloss:0.06307
[3965]	validation_0-logloss:0.01536	validation_1-logloss:0.06306
[3966]	validation_0-logloss:0.01536	validation_1-logloss:0.06306
[3967]	validation_0-logloss:0.01536	validation_1-logloss:0.06306
[3968]	validation_0-loglo

[4080]	validation_0-logloss:0.01472	validation_1-logloss:0.06297
[4081]	validation_0-logloss:0.01472	validation_1-logloss:0.06297
[4082]	validation_0-logloss:0.01472	validation_1-logloss:0.06297
[4083]	validation_0-logloss:0.01471	validation_1-logloss:0.06297
[4084]	validation_0-logloss:0.01471	validation_1-logloss:0.06297
[4085]	validation_0-logloss:0.01470	validation_1-logloss:0.06297
[4086]	validation_0-logloss:0.01469	validation_1-logloss:0.06297
[4087]	validation_0-logloss:0.01469	validation_1-logloss:0.06296
[4088]	validation_0-logloss:0.01468	validation_1-logloss:0.06297
[4089]	validation_0-logloss:0.01468	validation_1-logloss:0.06296
[4090]	validation_0-logloss:0.01467	validation_1-logloss:0.06297
[4091]	validation_0-logloss:0.01467	validation_1-logloss:0.06296
[4092]	validation_0-logloss:0.01467	validation_1-logloss:0.06297
[4093]	validation_0-logloss:0.01466	validation_1-logloss:0.06296
[4094]	validation_0-logloss:0.01466	validation_1-logloss:0.06296
[4095]	validation_0-loglo

[4207]	validation_0-logloss:0.01392	validation_1-logloss:0.06282
[4208]	validation_0-logloss:0.01392	validation_1-logloss:0.06282
[4209]	validation_0-logloss:0.01392	validation_1-logloss:0.06282
[4210]	validation_0-logloss:0.01391	validation_1-logloss:0.06282
[4211]	validation_0-logloss:0.01391	validation_1-logloss:0.06281
[4212]	validation_0-logloss:0.01390	validation_1-logloss:0.06281
[4213]	validation_0-logloss:0.01390	validation_1-logloss:0.06281
[4214]	validation_0-logloss:0.01389	validation_1-logloss:0.06281
[4215]	validation_0-logloss:0.01389	validation_1-logloss:0.06281
[4216]	validation_0-logloss:0.01388	validation_1-logloss:0.06281
[4217]	validation_0-logloss:0.01388	validation_1-logloss:0.06281
[4218]	validation_0-logloss:0.01388	validation_1-logloss:0.06281
[4219]	validation_0-logloss:0.01388	validation_1-logloss:0.06281
[4220]	validation_0-logloss:0.01387	validation_1-logloss:0.06281
[4221]	validation_0-logloss:0.01387	validation_1-logloss:0.06281
[4222]	validation_0-loglo

[4334]	validation_0-logloss:0.01338	validation_1-logloss:0.06285
[4335]	validation_0-logloss:0.01338	validation_1-logloss:0.06285
[4336]	validation_0-logloss:0.01337	validation_1-logloss:0.06284
[4337]	validation_0-logloss:0.01337	validation_1-logloss:0.06285
[4338]	validation_0-logloss:0.01336	validation_1-logloss:0.06285
[4339]	validation_0-logloss:0.01336	validation_1-logloss:0.06285
[4340]	validation_0-logloss:0.01334	validation_1-logloss:0.06284
[4341]	validation_0-logloss:0.01333	validation_1-logloss:0.06284
[4342]	validation_0-logloss:0.01333	validation_1-logloss:0.06284
[4343]	validation_0-logloss:0.01333	validation_1-logloss:0.06284
[4344]	validation_0-logloss:0.01332	validation_1-logloss:0.06284
[4345]	validation_0-logloss:0.01332	validation_1-logloss:0.06283
[4346]	validation_0-logloss:0.01331	validation_1-logloss:0.06284
[4347]	validation_0-logloss:0.01331	validation_1-logloss:0.06284
[4348]	validation_0-logloss:0.01330	validation_1-logloss:0.06285
[4349]	validation_0-loglo

In [119]:
print("ROC-AUC Score: ", np.mean(roc_scores))

ROC-AUC Score:  0.7327445663893913


In [157]:
pred = []
for model in Models:
    pred.append(model.predict_proba(test.drop(['order_id'], axis = 1)))

print(np.array(pred).shape)
print(np.mean(pred, axis = 0).shape)
pred = np.argmax(np.mean(pred, axis = 0), axis = 1)

/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pa

(10, 144844, 2)
(144844, 2)


In [158]:
pred.shape

(144844,)

In [159]:
pred_file = {'order_id': test['order_id'], 'cancelled': pred}
pred_file = pd.DataFrame(pred_file)
pred_file.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [160]:
pred_file.to_csv('Sub1.csv', index = False)